# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports of packages:

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import datetime
from datetime import date, timedelta
from plotly.subplots import make_subplots
from unittest import skip
import chart_studio
import chart_studio.plotly as py


# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject
import pandas as pd
import requests

Import of datasets:

In [2]:
url_historik = 'https://github.com/NumEconCopenhagen/projects-2023-siuuu/blob/main/dataproject/Kurshistorik%2C%20safemax.xlsx?raw=true'
response_historik = requests.get(url_historik)
content_historik = response_historik.content
# Creating dataframe from excel file that contains the historical data
DataSafeMax = pd.read_excel(content_historik, sheet_name='KursHistorisk', usecols="B:G", skiprows=2, engine='openpyxl')
DataSafeMax = DataSafeMax.rename(columns={'DMax': 'Dmax'})
# Creating dataframe from excel file that contains the historical data
Kurserhistorik = pd.read_excel(content_historik, sheet_name='KurserHistorisk', engine='openpyxl')
Kurserhistorik.drop(Kurserhistorik.columns[0], axis=1, inplace=True)
# Now we want to create a new dataframe that contains the historical data for the funds, why we import a nwe excel file
url_indeks = 'https://github.com/NumEconCopenhagen/projects-2023-siuuu/blob/main/dataproject/IndeksData_udenmakro.xlsx'
response_indeks = requests.get(url_indeks)
content_indeks = response_indeks.content

url_indeks = 'https://github.com/NumEconCopenhagen/projects-2023-siuuu/blob/main/dataproject/IndeksData_udenmakro.xlsx?raw=true'
response_indeks = requests.get(url_indeks)
content_indeks = response_indeks.content
# The dataframe is now created
IndeksData = pd.read_excel(content_indeks, sheet_name='Fonde', usecols="C,E, G:H, AB:AC, HM:HN, HQ:HR", skiprows=1, engine='openpyxl')


In [3]:
# Now we remove data from the historical data that is already included in the SafeMax data
Kurserhistorik = Kurserhistorik[~Kurserhistorik.Dato.isin(DataSafeMax.Dato)]
Kurserhistorik = Kurserhistorik[Kurserhistorik['Dato'] < '2021-12-31']

In [4]:
# Now we merge the two dataframes
dato_data = pd.DataFrame(pd.date_range(start='2012-10-01', end='today'), columns=['Dato'])

FinalData = pd.merge(Kurserhistorik, DataSafeMax, how='outer', sort=True).fillna(method="ffill")
FinalData = pd.merge(FinalData, dato_data, how='outer', on='Dato', sort=True).fillna(method="ffill")

In [5]:
# Now we want to create a new dataframe that contains the data of the optimal portfolio and the safe portfolio, and remove the cells that contain NA.
Optimum = FinalData.drop(columns=['Safe', 'Max', 'Responsible', 'Dmax']).dropna()

# Removing the beginning of data, because it contains the same data.
Optimum = Optimum.iloc[6:]
Today = date.today()

In [9]:
#Fjerner blanks ift. datoer i fremtiden
FinalData = FinalData[FinalData['Dato'] < datetime.datetime.now()]

#Laver liste med d. 30-12 i hvert år på nær seneste år, der er d. 31
FirstDateOfYear = ['2013-12-30', '2014-12-30', '2015-12-30', '2016-12-30',
               '2017-12-30', '2018-12-30', '2019-12-30', '2020-12-30',
               '2021-12-30', '2022-12-30', '2023-12-30', '2024-12-30',
               '2025-12-30', '2026-12-30', '2027-12-30', '2028-12-30']
FirstDateOfYear = pd.to_datetime(FirstDateOfYear)

In [15]:
#Benchmarks
NykreditData = IndeksData.iloc[:, 0:2].dropna()
MSCIWorldData = IndeksData.iloc[:, 2:4].dropna()
CIBOR = IndeksData.iloc[:, 4:6].dropna()

#Rename Columns
CIBOR = CIBOR.rename(columns = {CIBOR.columns[0]: 'Dato', CIBOR.columns[1]: 'CIBOR'})
NykreditData = NykreditData.rename(columns = {NykreditData.columns[0]: 'Dato', NykreditData.columns[1]: 'Nykredit'})
MSCIWorldData = MSCIWorldData.rename(columns = {MSCIWorldData.columns[0]: 'Dato', MSCIWorldData.columns[1]: 'MSCIWorld'})

#We are merging Benchmarks dataframes together 
Benchmarks = pd.merge(Optimum, pd.merge(pd.merge(MSCIWorldData, CIBOR, how='outer', on='Dato').fillna(method='ffill'), NykreditData, how='outer', on='Dato').fillna(method='ffill'), how='left', on='Dato').fillna(method='ffill')

#Calculating the Std 
Benchmarks["Optimum daily return"]=Benchmarks["Optimum"].pct_change(1)
Benchmarks["Nykredit daily return"]=Benchmarks["Nykredit"].pct_change(1)
Benchmarks["CIBOR daily return"]=Benchmarks["CIBOR"].pct_change(1)
Benchmarks["MSCIWorld daily return"]=Benchmarks["MSCIWorld"].pct_change(1)

Benchmarks["NykreditIndeks"] = (Benchmarks["Nykredit"]/Benchmarks["Nykredit"].iloc[0])*100
Benchmarks["OptimumIndeks"] = (Benchmarks["Optimum"]/Benchmarks["Optimum"].iloc[0])*100
Benchmarks["Optimum Std"]=Benchmarks["Optimum daily return"].rolling(365).std()*(365**0.5)
Benchmarks["Nykredit Std"]=Benchmarks["Nykredit daily return"].rolling(365).std()*(365**0.5)
Benchmarks["MSCIWorldIndeks"] = (Benchmarks["MSCIWorld"]/Benchmarks["MSCIWorld"].iloc[0])*100
Benchmarks["MSCI World Std"]=Benchmarks["MSCIWorld daily return"].rolling(365).std()*(365**0.5)
Benchmarks["CIBORIndeks"] = (Benchmarks["CIBOR"]/Benchmarks["CIBOR"].iloc[0])*100
Benchmarks["CIBOR Std"]=Benchmarks["CIBOR daily return"].rolling(365).std()*(365**0.5)

In [39]:
#Der benyttes kun data efter 2012-10-01 i præsentationsmaterialet, men data er større, irrelevant data fjernes.
Optimum = Optimum[Optimum['Dato'] >= '2010-09-01']

#Danner sammenligning ml optimum og 50-50 benchmark
FiftyFifty = Benchmarks.copy()

FiftyFifty = FiftyFifty[FiftyFifty['Dato'] >= '2010-09-30'].reset_index(drop=True)

#Følgende iteration laver rebalancering for 50-50 benchmark ift. Optimum
for ind, row in FiftyFifty.iterrows():
    if FiftyFifty.loc[ind, "Dato"] in FirstDateOfYear :
        FiftyFifty.loc[ind, "Løbende vægt Nykredit"] = 50 * (1 + FiftyFifty.loc[ind, "Nykredit daily return"])
        FiftyFifty.loc[ind, "Løbende vægt MSCI World"] = 50 * (1 + FiftyFifty.loc[ind, "MSCIWorld daily return"])
        FiftyFifty.loc[ind, "Vægtsum"] = FiftyFifty.loc[ind, "Løbende vægt Nykredit"] + FiftyFifty.loc[ind, "Løbende vægt MSCI World"]
        FiftyFifty.loc[ind, "Ændring vægtsum"] = (FiftyFifty.loc[ind, "Vægtsum"]/100)-1
        FiftyFifty.loc[ind, "Indeks 100"] = FiftyFifty.loc[ind - 1, "Indeks 100"] * (1 + FiftyFifty.loc[ind, "Ændring vægtsum"])
    else :
        if ind == 0 :
            FiftyFifty.loc[ind, "Løbende vægt Nykredit"] = 50
            FiftyFifty.loc[ind, "Løbende vægt MSCI World"] = 50 
            FiftyFifty.loc[ind, "Vægtsum"] = FiftyFifty.loc[ind, "Løbende vægt Nykredit"] + FiftyFifty.loc[ind, "Løbende vægt MSCI World"]
            FiftyFifty.loc[ind, "Ændring vægtsum"] = np.NaN
            FiftyFifty.loc[ind, "Indeks 100"] = 100
        else : 
            FiftyFifty.loc[ind, "Løbende vægt Nykredit"] = FiftyFifty.loc[ind-1, "Løbende vægt Nykredit"] * (1 + FiftyFifty.loc[ind, "Nykredit daily return"])
            FiftyFifty.loc[ind, "Løbende vægt MSCI World"] = FiftyFifty.loc[ind-1, "Løbende vægt MSCI World"] * (1 + FiftyFifty.loc[ind, "MSCIWorld daily return"])
            FiftyFifty.loc[ind, "Vægtsum"] = FiftyFifty.loc[ind, "Løbende vægt Nykredit"] + FiftyFifty.loc[ind, "Løbende vægt MSCI World"]
            FiftyFifty.loc[ind, "Ændring vægtsum"] = (FiftyFifty.loc[ind, "Vægtsum"]/FiftyFifty.loc[ind-1, "Vægtsum"])-1
            FiftyFifty.loc[ind, "Indeks 100"] = FiftyFifty.loc[ind - 1, "Indeks 100"] * (1+ FiftyFifty.loc[ind, "Ændring vægtsum"])

#Fjerner irrelevante datoer
FiftyFifty = FiftyFifty[FiftyFifty['Dato'] <= MSCIWorldData["Dato"].iloc[-1]]

#Beregner indeks for Optimum
FiftyFifty["OptimumIndeks"] = (FiftyFifty["Optimum"]/FiftyFifty["Optimum"].iloc[0])*100

#Beregner standardafvigelse for Optimum og Benchmark
FiftyFifty["Optimum Std"]=FiftyFifty["Optimum daily return"].rolling(365).std()*(365**0.5)
FiftyFifty["Benchmark Std"]=FiftyFifty["Ændring vægtsum"].rolling(365).std()*(365**0.5)

In [32]:
#Definerer måned og år for hver række
FiftyFifty['Maned'] = FiftyFifty["Dato"].dt.to_period('M')

current_month = datetime.datetime.now().strftime("%B")
if current_month == 'January':
    month_value = -3
else:
    month_value = -2
#Fjerner duplicates og beholder sidste (får sidste observation for hver måned)
FiftyFiftyFirst_Dec_30 = FiftyFifty[((FiftyFifty["Dato"].dt.month == 12) & (FiftyFifty["Dato"].dt.day == 30))]
FiftyFiftyFirst = FiftyFifty.copy()
FiftyFiftyFirst = FiftyFiftyFirst.drop_duplicates(subset=["Maned"], keep='last', inplace=False, ignore_index=False)

#Beregner månedligt afkast for optimum, benchmark
FiftyFiftyFirst["Optimum maned afkast"]= FiftyFiftyFirst["Optimum"].pct_change(1)
FiftyFiftyFirst["Benchmark maned afkast"]= FiftyFiftyFirst["Indeks 100"].pct_change(1)

#Beregner månedlig CIBOR
FiftyFiftyFirst["CIBOR"]=(FiftyFiftyFirst["CIBOR"]/100)/12

#Beregner mer' afkast for benchmark og optimum
FiftyFiftyFirst["Benchmark mer maned afkast"]=FiftyFiftyFirst["Benchmark maned afkast"]-FiftyFiftyFirst["CIBOR"]
FiftyFiftyFirst["Optimum mer maned afkast"]=FiftyFiftyFirst["Optimum maned afkast"]-FiftyFiftyFirst["CIBOR"]

#Sætter første observation lig kursudvikling præs (beregnet på historisk data)
FiftyFiftyFirst.iloc[0, FiftyFiftyFirst.columns.get_loc('Optimum mer maned afkast')] = -0.006226/100
FiftyFiftyFirst.iloc[0, FiftyFiftyFirst.columns.get_loc('Benchmark mer maned afkast')] = -0.65/100

#OBS skal nedenstående skal kun benyttes hvis månedsskiftet ikke er indgået
FiftyFiftyFirst = FiftyFiftyFirst[:-1]

FiftyFiftyFirst = FiftyFiftyFirst[1:]

#Beregner gns månedligt afkast for benchmark og optimum
MiddelAfkastBM5050=FiftyFiftyFirst["Benchmark mer maned afkast"].mean()
MiddelAfkastOptimum5050=FiftyFiftyFirst["Optimum mer maned afkast"].mean()

#Beregner standardafvigelse for benchmark og optimum
StdAfkastOptimum5050=FiftyFiftyFirst["Optimum mer maned afkast"].std()*(12**0.5)
StdAfkastBM5050 = FiftyFiftyFirst["Benchmark mer maned afkast"].std()*(12**0.5)

#Beregner sharpe ratio for benchmark og optimum
SharpeOptimum5050=(MiddelAfkastOptimum5050/FiftyFiftyFirst["Optimum mer maned afkast"].std())*(12**0.5)
SharpeBM5050=(MiddelAfkastBM5050/FiftyFiftyFirst["Benchmark mer maned afkast"].std())*(12**0.5)

FiftyFiftyFirst = pd.concat([FiftyFiftyFirst_Dec_30, FiftyFiftyFirst], axis=0).sort_values('Dato').reset_index(drop=True)

#SKAL ÆNDRES TIL -2 I FEBRUAR
FiftyFiftyFirstATD = FiftyFifty.resample('Y', on='Dato')['Dato'].agg(['last']).iat[month_value, 0] - timedelta(days=1)
OptimumFirstATD = FiftyFifty[FiftyFifty['Dato'] == FiftyFiftyFirstATD]
FiftyFiftyFirstATD_31 = FiftyFifty.resample('Y', on='Dato')['Dato'].agg(['last']).iat[month_value, 0]
OptimumFirstATD_31 = FiftyFifty[FiftyFifty['Dato'] == FiftyFiftyFirstATD_31]

#Beregner afkast År-til-dato
AfkastATD5050Optimum = (FiftyFiftyFirst.iloc[-1, FiftyFiftyFirst.columns.get_loc('Optimum')]/OptimumFirstATD_31.iloc[0, OptimumFirstATD_31.columns.get_loc('Optimum')])-1
AfkastATD5050BM = (FiftyFiftyFirst.iloc[-1, FiftyFiftyFirst.columns.get_loc('Indeks 100')]/OptimumFirstATD.iloc[0, OptimumFirstATD.columns.get_loc('Indeks 100')])-1

#Beregner afkast for hele perioden
AfkastTotal5050Optimum = (FiftyFiftyFirst.iloc[-1, FiftyFiftyFirst.columns.get_loc('Optimum')]/FiftyFifty.iloc[0, FiftyFifty.columns.get_loc('Optimum')])-1
AfkastTotal5050BM = (FiftyFiftyFirst.iloc[-1, FiftyFiftyFirst.columns.get_loc('Indeks 100')]/100)-1




In [45]:
def kurserplot(FigurNavn, DatoSoejle, ArthaFond, FondsNavn, BMFond, BMNavn, StdFond, StdBM):
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=DatoSoejle, y=ArthaFond, mode='lines', line_color='#AD9E74', line_width=1.5, name=FondsNavn, fill= 'tozeroy', fillcolor='rgba(173, 158, 116, 0.05)'))
    fig.add_trace(go.Scatter(x=DatoSoejle, y=BMFond, mode='lines', line_color='#131D2F', line_width=1.5, name=BMNavn, fill= 'tozeroy', fillcolor='rgba(173, 158, 116, 0.05)'))
    fig.update_xaxes(title_text='', showgrid=False)

    ymax = max([ArthaFond.max(), BMFond.max()]) + 10
    ymin = min([ArthaFond.min(), BMFond.min()]) - 10

    fig.update_yaxes(title_text='', range=[ymin, ymax], gridcolor="#F0F0F0")

    fig.update_layout(
    plot_bgcolor="white",
    font_color="black",
    title=dict(text = "Afkast periode: " + DatoSoejle.iloc[365].strftime("%d-%m-%Y") + " til " + DatoSoejle.iloc[-1].strftime("%d-%m-%Y"), font=dict(size=16)),
    xaxis_range=[DatoSoejle.iloc[0], DatoSoejle.iloc[-1]],
    margin=dict(t=30,l=0,b=10,r=10),
    width=600,
    height=450,
    #xaxis=dict(tickformat="%Y"),
    font_family="Grandview",
    font = dict(
        size = 16
    ),
    legend=dict(
    orientation="h"
    )
    )
    fig.show()
    
    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=DatoSoejle, y=StdFond, mode='lines', line_color='#AD9E74',
                     line_width=1.5, name=FondsNavn, fill= 'tozeroy', fillcolor='rgba(173, 158, 116, 0.05)'))
    fig2.add_trace(go.Scatter(x=DatoSoejle, y=StdBM, mode='lines', line_color='#131D2F',
                     line_width=1.5, name=BMNavn, fill= 'tozeroy', fillcolor='rgba(173, 158, 116, 0.05)'))
    fig2.update_xaxes(title_text='', showgrid=False)
    fig2.update_yaxes(title_text='', gridcolor="#F0F0F0")
    fig2.update_layout(
        plot_bgcolor="white",
        font_color="black",
        #title=dict(text = "Risiko periode: " + str(DatoSoejle.iloc[365])[0:7] + " til " + (datetime.datetime.now() - datetime.timedelta(weeks=4)).strftime("%d-%m-%Y"), font=dict(size=16)),
        title=dict(text = "Risiko periode: " + DatoSoejle.iloc[365].strftime("%d-%m-%Y") + " til " + DatoSoejle.iloc[-1].strftime("%d-%m-%Y"), font=dict(size=16)),
        xaxis_range=[DatoSoejle.iloc[365], DatoSoejle.iloc[-1]],
        yaxis_tickformat = '.0%',margin=dict(t=30,l=0,b=10,r=10),
        #xaxis=dict(tickformat="%Y"),
        width=600,
        height=450,
        font_family="Grandview",
        font = dict(
            size = 16
        ),
        legend=dict(
        orientation="h"
    )
    )
    fig2.show()

now = datetime.datetime.now().date()
eop = now.replace(day=1)
eop = str(eop)

FiftyFifty = FiftyFifty[FiftyFifty['Dato'] < eop]


kurserplot("OptimumGraf", FiftyFifty['Dato'], FiftyFifty['OptimumIndeks'], "Optimum", FiftyFifty['Indeks 100'], "50/50% realkreditobl./globale aktier", FiftyFifty['Optimum Std'], FiftyFifty['Benchmark Std'])


In [47]:
colors = ['white', 'white', '#F0F0F0']
data = {'Color' : colors}
df = pd.DataFrame(data)

ValuesOpti=[["Optimum", "50/50% realkreditobl./globale aktier", "<b>Difference</b>"],
        ["{:.1%}".format(round(AfkastTotal5050Optimum,3)),"{:.1%}".format(round(AfkastTotal5050BM,3)), "{:.1%}".format(round(AfkastTotal5050Optimum-AfkastTotal5050BM,3))],
        ["{:.1%}".format(round(AfkastATD5050Optimum,3)), "{:.1%}".format(round(AfkastATD5050BM,3)),  "{:.1%}".format(round(AfkastATD5050Optimum-AfkastATD5050BM,3))],
        ["{:.1%}".format(round(StdAfkastOptimum5050,3)), "{:.1%}".format(round(StdAfkastBM5050,3)),  "{:.1%}".format(round(StdAfkastOptimum5050-StdAfkastBM5050,3))],
        ["{:.2f}".format(round(SharpeOptimum5050, 2)), "{:.2f}".format(round(SharpeBM5050, 2)), "{:.2f}".format(round(SharpeOptimum5050-SharpeBM5050, 2))]]


def Table(FigurNavn, Values):
    Header=["<b>Afkast for periode</b>", "<b>Afkast siden start</b>", "<b>Afkast 2023</b>", "<b>Risiko</b>", "<b>Sharpe</b>"]
    TableFig = go.Figure(data=[go.Table(
        columnwidth = [310,160,120,80,80],
        header=dict(
        values=Header, 
        align=['left','center'],
        fill_color="white",
        font=dict(color='black', size=16)
        ),
        cells=dict(values=Values, 
        align=['left','center'],
        height=30,
        line_color=[df.Color],
        fill_color=[df.Color],
        font=dict(color='black', size=16)))])
    TableFig.show()
    TableFig.update_layout(width=700, height=150, margin={"l":0,"r":0,"t":0,"b":0}, font_family="Grandview")


Table("OptimumTest", ValuesOpti)


# Conclusion

ADD CONCISE CONLUSION.